In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import Qdrant
from langchain_core.runnables import RunnablePassthrough 

# ✅ 올바른 임포트 방법 (여러 옵션 시도)
try:
    from langchain.retrievers import EnsembleRetriever
    print("✅ EnsembleRetriever imported from langchain.retrievers")
except ImportError:
    try:
        from langchain_community.retrievers import EnsembleRetriever
        print("✅ EnsembleRetriever imported from langchain_community.retrievers")
    except ImportError:
        # 수동으로 EnsembleRetriever 구현
        print("⚠️ EnsembleRetriever not found. Using manual implementation.")
        from langchain_core.retrievers import BaseRetriever
        from langchain_core.documents import Document
        from typing import List
        from collections import defaultdict
        
        class EnsembleRetriever(BaseRetriever):
            retrievers: List[BaseRetriever]
            weights: List[float]
            c: int = 60
            
            def _get_relevant_documents(self, query: str) -> List[Document]:
                # 각 retriever에서 문서 검색
                doc_lists = [retriever.get_relevant_documents(query) 
                           for retriever in self.retrievers]
                
                # RRF (Reciprocal Rank Fusion) 스코어 계산
                rrf_score = defaultdict(float)
                for doc_list, weight in zip(doc_lists, self.weights):
                    for rank, doc in enumerate(doc_list, start=1):
                        rrf_score[doc.page_content] += weight / (rank + self.c)
                
                # 스코어 기준으로 정렬
                all_docs = {doc.page_content: doc for doc_list in doc_lists for doc in doc_list}
                sorted_docs = sorted(rrf_score.items(), key=lambda x: x[1], reverse=True)
                
                return [all_docs[content] for content, _ in sorted_docs]

from qdrant_client import QdrantClient # 🌟 Qdrant 클라이언트 임포트 🌟

# ----------------------------------------------------
# 0. 환경 설정
# ----------------------------------------------------

# ⚠️ 보안 경고: 실제 환경에서는 환경 변수 파일(.env)을 사용하세요.
os.environ["OPENAI_API_KEY"] = "."

# ----------------------------------------------------
# 1. 환경 설정 및 클라이언트 초기화
# ----------------------------------------------------

# (1) Qdrant 연결 정보 설정
QDRANT_HOST = "localhost" 
QDRANT_PORT = 6333        

# (2) OpenAI Embeddings 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 🌟🌟🌟 (3) Qdrant VectorStore 초기화 로직 수정 🌟🌟🌟
# 3-1. QdrantClient 인스턴스 생성
client = QdrantClient(
    url=f"http://{QDRANT_HOST}:{QDRANT_PORT}" 
)

# ----------------------------------------------------
# 2. 두 개의 Retriever 생성
# ----------------------------------------------------

# (1) 명소 컬렉션 검색기 (기존 컬렉션)
ATTRACTION_COLLECTION = "seoul-attraction-media"
qdrant_attraction = Qdrant(
    client=client, 
    collection_name=ATTRACTION_COLLECTION,
    embeddings=embeddings,
)
retriever_attraction = qdrant_attraction.as_retriever(search_kwargs={"k": 3})

# (2) 레스토랑 컬렉션 검색기 (새로 추가된 컬렉션)
RESTAURANT_COLLECTION = "seoul-restaurant"
qdrant_restaurant = Qdrant(
    client=client, 
    collection_name=RESTAURANT_COLLECTION,
    embeddings=embeddings,
)
retriever_restaurant = qdrant_restaurant.as_retriever(search_kwargs={"k": 3}) # 동일하게 3개 검색

print(f"✅ Qdrant 컬렉션 '{ATTRACTION_COLLECTION}' 및 '{RESTAURANT_COLLECTION}' 검색기 준비 완료.")

# ----------------------------------------------------
# 3. 앙상블 검색기로 통합 (EnsembleRetriever)
# ----------------------------------------------------

# 두 검색기를 통합합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever_attraction, retriever_restaurant],
    weights=[0.5, 0.5] # 각 검색 결과에 50%의 가중치를 부여합니다.
)

print("✅ 앙상블 검색기 생성 완료 (명소 + 레스토랑)")

# ----------------------------------------------------
# 4. RAG 프롬프트 정의
# ----------------------------------------------------

template = """
You are a travel guide chatbot specialized in Korean attractions integrated with K-content (K-dramas, movies, K-pop, and fictional universes).
Your task is to recommend Korean locations by connecting them with real K-dramas or, if unavailable, by inferring plausible K-content relevance based on general knowledge.

Use the 'reference context' information FIRST when available.
If the reference context does not contain direct K-content information for a location, infer potential connections based on:
- atmosphere
- architecture style
- cultural vibe
- similarities to scenes in well-known dramas or movies
- aesthetics common in K-pop music videos or fantasy universes

IMPORTANT:
Do NOT fabricate specific filming facts.
When inferring connections, use expressions such as:
- "This location has a mood similar to scenes from..."
- "Its atmosphere resembles places featured in..."
- "This fits the visual style often used in K-pop music videos..."

Your answer MUST follow this structured format for each attraction:

📍 **[Attraction Name]**
- **Location:** …
- **Main Features:** …
- **K-Content Connection:** …
- **Recommended For:** …
- **Nearby Spots (optional):** …

Also, please briefly recommend and explain restaurants near the attraction
==========================
REFERENCE CONTEXT (K-DRAMA LOCATIONS + KPOP DEMON HUNTERS LOCATIONS)
==========================

{{
  "dramas": [
    {{
      "title": "Goblin",
      "year": 2016,
      "locations": [
        {{
          "name": "Samcheong-dong Street",
          "address": "Jongno-gu, Seoul",
          "description": "Frequently appeared as the walking path of Goblin and Eun-tak.",
          "scene": "Emotional walking and summoning scenes."
        }},
        {{
          "name": "Deoksugung Stone Wall Road",
          "address": "Jung-gu, Seoul",
          "description": "A key location for several emotional scenes.",
          "scene": "Memorable scenes between Goblin and Eun-tak."
        }}
      ]
    }},
    {{
      "title": "Itaewon Class",
      "year": 2020,
      "locations": [
        {{
          "name": "Itaewon Global Food Street",
          "address": "Yongsan-gu, Seoul",
          "description": "The real-life area that inspired the business district in the drama.",
          "scene": "Main street where Park Saeroyi operates."
        }},
        {{
          "name": "Noksapyeong Station Area",
          "address": "Yongsan-gu, Seoul",
          "description": "Known for night scenes and character dialogues.",
          "scene": "Conversation scenes between Saeroyi and Yi-seo."
        }}
      ]
    }},
    {{
      "title": "My Love From The Star",
      "year": 2013,
      "locations": [
        {{
          "name": "N Seoul Tower",
          "address": "Yongsan-gu, Seoul",
          "description": "A major romantic landmark.",
          "scene": "Date scenes between Cheon Song-yi and Do Min-joon."
        }},
        {{
          "name": "Dongdaemun Design Plaza (DDP)",
          "address": "Jung-gu, Seoul",
          "description": "Modern architecture featured in the drama.",
          "scene": "Commercial and media scenes with Cheon Song-yi."
        }}
      ]
    }},
    {{
      "title": "Crash Landing on You",
      "year": 2019,
      "locations": [
        {{
          "name": "Banpo Hangang Park (Jamsu Bridge area)",
          "address": "Seocho-gu, Seoul",
          "description": "Romantic riverside scenes.",
          "scene": "Scenes showing Yoon Se-ri's personal life."
        }},
        {{
          "name": "MBC Sangam",
          "address": "Mapo-gu, Seoul",
          "description": "Background for media and newsroom-related scenes.",
          "scene": "Broadcast and press scenes."
        }}
      ]
    }},
    {{
      "title": "Extraordinary Attorney Woo",
      "year": 2022,
      "locations": [
        {{
          "name": "Yeouido Hangang Park",
          "address": "Yeongdeungpo-gu, Seoul",
          "description": "Park scenes for rest and conversations.",
          "scene": "Woo Young-woo and colleagues walking scenes."
        }},
        {{
          "name": "Jonggak Area",
          "address": "Jongno-gu, Seoul",
          "description": "Surrounding streets of the fictional law firm.",
          "scene": "Exterior shots of business district."
        }}
      ]
    }}
  ],

  "kpop_demon_hunters": {{
    "main_locations": [
      {{
        "name": "Hongdae Walking Street",
        "address": "Mapo-gu, Seoul",
        "description": "Key cultural youth area used as a fictional location for supernatural events in the K-pop Demon Hunters universe."
      }},
      {{
        "name": "COEX Gangnam",
        "address": "Gangnam-gu, Seoul",
        "description": "Fictional headquarters setting in the Demon Hunters storyline."
      }},
      {{
        "name": "N Seoul Tower",
        "address": "Yongsan-gu, Seoul",
        "description": "A central point where dimensional rifts appear in the Demon Hunters universe."
      }}
    ]
  }}
}}

==========================
--- Reference Context ---
{context}

--- User Question ---
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

# ----------------------------------------------------
# 5. LLM (GPT) 및 RAG 체인 구축
# ----------------------------------------------------

# (1) Chat 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# (2) 검색 결과 포맷팅 함수
def format_docs(docs):
    # 검색된 문서들의 내용을 하나의 컨텍스트 문자열로 합칩니다.
    return "\n\n".join(doc.page_content for doc in docs)

# (3) RAG 체인 정의 (LangChain Expression Language 사용)
rag_chain = (
    # ensemble_retriever로 변경
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

print("✅ RAG 체인 구축 완료 (EnsembleRetriever -> Prompt -> LLM)")

# ----------------------------------------------------
# 6. 챗봇 테스트 함수
# ----------------------------------------------------

def run_chatbot(query):
    print("\n" + "="*50)
    print(f"🎤 사용자 질문: {query}")
    print("="*50)
    
    response = rag_chain.invoke(query)
    
    print("\n🤖 챗봇 답변:")
    print(response.content)
    print("="*50)
    
    return response.content

# ----------------------------------------------------
# 7. 테스트 실행 예시
# ----------------------------------------------------
# (Qdrant 서버가 실행 중이어야 합니다.)

# run_chatbot("창덕궁 주변의 K-드라마 촬영 장소에 대해 자세히 알려주세요.")
# run_chatbot("광희문은 어떤 드라마와 연관이 있으며, 방문 시 무엇을 볼 수 있나요?")
# run_chatbot("이태원 근처 맛집 추천해줘")

⚠️ EnsembleRetriever not found. Using manual implementation.
✅ Qdrant 컬렉션 'seoul-attraction-media' 및 'seoul-restaurant' 검색기 준비 완료.
✅ 앙상블 검색기 생성 완료 (명소 + 레스토랑)
✅ RAG 체인 구축 완료 (EnsembleRetriever -> Prompt -> LLM)


In [45]:
!pip install --upgrade langchain

  Attempting uninstall: langchain
    Found existing installation: langchain 1.0.5
    Uninstalling langchain-1.0.5:
      Successfully uninstalled langchain-1.0.5


In [46]:
pip install langchain>=0.1.0 langchain-community>=0.0.20

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import Qdrant
from langchain_core.runnables import RunnablePassthrough 

# ✅ EnsembleRetriever를 사용하지 않고 직접 구현
# 두 retriever의 결과를 합치는 함수
from langchain_core.documents import Document
from typing import List

def combine_retrievers(query: str, retrievers: List, weights: List[float] = None) -> List[Document]:
    """
    여러 retriever의 결과를 합치는 함수
    """
    if weights is None:
        weights = [1.0 / len(retrievers)] * len(retrievers)
    
    all_docs = []
    seen_contents = set()
    
    # 각 retriever에서 문서 검색
    for retriever, weight in zip(retrievers, weights):
        docs = retriever.invoke(query)
        for doc in docs:
            # 중복 제거
            if doc.page_content not in seen_contents:
                seen_contents.add(doc.page_content)
                all_docs.append(doc)
    
    return all_docs

print("✅ Custom retriever combination function loaded")

from qdrant_client import QdrantClient # 🌟 Qdrant 클라이언트 임포트 🌟

# ----------------------------------------------------
# 0. 환경 설정
# ----------------------------------------------------

# ⚠️ 보안 경고: 실제 환경에서는 환경 변수 파일(.env)을 사용하세요.
os.environ["OPENAI_API_KEY"] = "."

# ----------------------------------------------------
# 1. 환경 설정 및 클라이언트 초기화
# ----------------------------------------------------

# (1) Qdrant 연결 정보 설정
QDRANT_HOST = "localhost" 
QDRANT_PORT = 6333        

# (2) OpenAI Embeddings 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 🌟🌟🌟 (3) Qdrant VectorStore 초기화 로직 수정 🌟🌟🌟
# 3-1. QdrantClient 인스턴스 생성
client = QdrantClient(
    url=f"http://{QDRANT_HOST}:{QDRANT_PORT}" 
)

# ----------------------------------------------------
# 2. 두 개의 Retriever 생성
# ----------------------------------------------------

# (1) 명소 컬렉션 검색기 (기존 컬렉션)
ATTRACTION_COLLECTION = "seoul-attraction-media"
qdrant_attraction = Qdrant(
    client=client, 
    collection_name=ATTRACTION_COLLECTION,
    embeddings=embeddings,
)
retriever_attraction = qdrant_attraction.as_retriever(search_kwargs={"k": 3})

# (2) 레스토랑 컬렉션 검색기 (새로 추가된 컬렉션)
RESTAURANT_COLLECTION = "seoul-restaurant"
qdrant_restaurant = Qdrant(
    client=client, 
    collection_name=RESTAURANT_COLLECTION,
    embeddings=embeddings,
)
retriever_restaurant = qdrant_restaurant.as_retriever(search_kwargs={"k": 3}) # 동일하게 3개 검색

print(f"✅ Qdrant 컬렉션 '{ATTRACTION_COLLECTION}' 및 '{RESTAURANT_COLLECTION}' 검색기 준비 완료.")

# ----------------------------------------------------
# 3. 커스텀 검색 함수 (EnsembleRetriever 대체)
# ----------------------------------------------------

def multi_retriever_search(query: str) -> List[Document]:
    """
    두 개의 retriever 결과를 합치는 함수
    """
    # 각 retriever에서 검색
    docs_attraction = retriever_attraction.invoke(query)
    docs_restaurant = retriever_restaurant.invoke(query)
    
    # 결과 합치기 (중복 제거)
    all_docs = []
    seen_contents = set()
    
    for doc in docs_attraction + docs_restaurant:
        if doc.page_content not in seen_contents:
            seen_contents.add(doc.page_content)
            all_docs.append(doc)
    
    return all_docs

print("✅ 멀티 검색기 함수 준비 완료")

# ----------------------------------------------------
# 4. RAG 프롬프트 정의
# ----------------------------------------------------

template = """
You are a travel guide chatbot specialized in Korean attractions integrated with K-content (K-dramas, movies, K-pop, and fictional universes).
Your task is to recommend Korean locations by connecting them with real K-dramas or, if unavailable, by inferring plausible K-content relevance based on general knowledge.

Use the 'reference context' information FIRST when available.
If the reference context does not contain direct K-content information for a location, infer potential connections based on:
- atmosphere
- architecture style
- cultural vibe
- similarities to scenes in well-known dramas or movies
- aesthetics common in K-pop music videos or fantasy universes

IMPORTANT:
Do NOT fabricate specific filming facts.
When inferring connections, use expressions such as:
- "This location has a mood similar to scenes from..."
- "Its atmosphere resembles places featured in..."
- "This fits the visual style often used in K-pop music videos..."

Your answer MUST follow this structured format for each attraction:

📍 **[Attraction Name]**
- **Location:** …
- **Main Features:** …
- **K-Content Connection:** …
- **Recommended For:** …
- **Nearby Spots (optional):** …

Also, please briefly recommend and explain restaurants near the attraction
==========================
REFERENCE CONTEXT (K-DRAMA LOCATIONS + KPOP DEMON HUNTERS LOCATIONS)
==========================

{{
  "dramas": [
    {{
      "title": "Goblin",
      "year": 2016,
      "locations": [
        {{
          "name": "Samcheong-dong Street",
          "address": "Jongno-gu, Seoul",
          "description": "Frequently appeared as the walking path of Goblin and Eun-tak.",
          "scene": "Emotional walking and summoning scenes."
        }},
        {{
          "name": "Deoksugung Stone Wall Road",
          "address": "Jung-gu, Seoul",
          "description": "A key location for several emotional scenes.",
          "scene": "Memorable scenes between Goblin and Eun-tak."
        }}
      ]
    }},
    {{
      "title": "Itaewon Class",
      "year": 2020,
      "locations": [
        {{
          "name": "Itaewon Global Food Street",
          "address": "Yongsan-gu, Seoul",
          "description": "The real-life area that inspired the business district in the drama.",
          "scene": "Main street where Park Saeroyi operates."
        }},
        {{
          "name": "Noksapyeong Station Area",
          "address": "Yongsan-gu, Seoul",
          "description": "Known for night scenes and character dialogues.",
          "scene": "Conversation scenes between Saeroyi and Yi-seo."
        }}
      ]
    }},
    {{
      "title": "My Love From The Star",
      "year": 2013,
      "locations": [
        {{
          "name": "N Seoul Tower",
          "address": "Yongsan-gu, Seoul",
          "description": "A major romantic landmark.",
          "scene": "Date scenes between Cheon Song-yi and Do Min-joon."
        }},
        {{
          "name": "Dongdaemun Design Plaza (DDP)",
          "address": "Jung-gu, Seoul",
          "description": "Modern architecture featured in the drama.",
          "scene": "Commercial and media scenes with Cheon Song-yi."
        }}
      ]
    }},
    {{
      "title": "Crash Landing on You",
      "year": 2019,
      "locations": [
        {{
          "name": "Banpo Hangang Park (Jamsu Bridge area)",
          "address": "Seocho-gu, Seoul",
          "description": "Romantic riverside scenes.",
          "scene": "Scenes showing Yoon Se-ri's personal life."
        }},
        {{
          "name": "MBC Sangam",
          "address": "Mapo-gu, Seoul",
          "description": "Background for media and newsroom-related scenes.",
          "scene": "Broadcast and press scenes."
        }}
      ]
    }},
    {{
      "title": "Extraordinary Attorney Woo",
      "year": 2022,
      "locations": [
        {{
          "name": "Yeouido Hangang Park",
          "address": "Yeongdeungpo-gu, Seoul",
          "description": "Park scenes for rest and conversations.",
          "scene": "Woo Young-woo and colleagues walking scenes."
        }},
        {{
          "name": "Jonggak Area",
          "address": "Jongno-gu, Seoul",
          "description": "Surrounding streets of the fictional law firm.",
          "scene": "Exterior shots of business district."
        }}
      ]
    }}
  ],

  "kpop_demon_hunters": {{
    "main_locations": [
      {{
        "name": "Hongdae Walking Street",
        "address": "Mapo-gu, Seoul",
        "description": "Key cultural youth area used as a fictional location for supernatural events in the K-pop Demon Hunters universe."
      }},
      {{
        "name": "COEX Gangnam",
        "address": "Gangnam-gu, Seoul",
        "description": "Fictional headquarters setting in the Demon Hunters storyline."
      }},
      {{
        "name": "N Seoul Tower",
        "address": "Yongsan-gu, Seoul",
        "description": "A central point where dimensional rifts appear in the Demon Hunters universe."
      }}
    ]
  }}
}}

==========================
--- Reference Context ---
{context}

--- User Question ---
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

# ----------------------------------------------------
# 5. LLM (GPT) 및 RAG 체인 구축
# ----------------------------------------------------

# (1) Chat 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# (2) 검색 결과 포맷팅 함수
def format_docs(docs):
    # 검색된 문서들의 내용을 하나의 컨텍스트 문자열로 합칩니다.
    return "\n\n".join(doc.page_content for doc in docs)

# (3) RAG 체인 정의 (커스텀 검색 함수 사용)
rag_chain = (
    # multi_retriever_search 함수로 검색 후 포맷팅
    {"context": lambda q: format_docs(multi_retriever_search(q)), "question": RunnablePassthrough()}
    | prompt
    | llm
)

print("✅ RAG 체인 구축 완료 (Multi-Retriever -> Prompt -> LLM)")

# ----------------------------------------------------
# 6. 챗봇 테스트 함수
# ----------------------------------------------------

def run_chatbot(query):
    print("\n" + "="*50)
    print(f"🎤 사용자 질문: {query}")
    print("="*50)
    
    response = rag_chain.invoke(query)
    
    print("\n🤖 챗봇 답변:")
    print(response.content)
    print("="*50)
    
    return response.content

# ----------------------------------------------------
# 7. 테스트 실행 예시
# ----------------------------------------------------
# (Qdrant 서버가 실행 중이어야 합니다.)

# run_chatbot("창덕궁 주변의 K-드라마 촬영 장소에 대해 자세히 알려주세요.")
# run_chatbot("광희문은 어떤 드라마와 연관이 있으며, 방문 시 무엇을 볼 수 있나요?")
# run_chatbot("이태원 근처 맛집 추천해줘")

✅ Custom retriever combination function loaded
✅ Qdrant 컬렉션 'seoul-attraction-media' 및 'seoul-restaurant' 검색기 준비 완료.
✅ 멀티 검색기 함수 준비 완료
✅ RAG 체인 구축 완료 (Multi-Retriever -> Prompt -> LLM)


In [52]:
run_chatbot("Can you introduce me to Korean kpop restaurants?")


🎤 사용자 질문: Can you introduce me to Korean kpop restaurants?

🤖 챗봇 답변:
Sure! Here are some popular K-pop themed restaurants in Korea that fans often visit to enjoy delicious food while immersing themselves in the K-pop culture:

📍 **Cafe de Lulu**
- **Location:** Gangnam-gu, Seoul
- **Main Features:** This cafe is decorated with K-pop memorabilia, including posters, albums, and merchandise from various K-pop groups. It has a cozy atmosphere perfect for fans to hang out.
- **K-Content Connection:** The cafe often plays K-pop music videos and hosts events related to popular idols, making it a great spot for fans to connect with their favorite content.
- **Recommended For:** K-pop fans looking for a themed dining experience and a place to socialize with fellow enthusiasts.
- **Nearby Spots:** COEX Mall, a popular shopping and entertainment complex.

📍 **K-Star Road**
- **Location:** Gangnam-gu, Seoul
- **Main Features:** This road features a series of K-pop themed cafes and restaurants, ea

'Sure! Here are some popular K-pop themed restaurants in Korea that fans often visit to enjoy delicious food while immersing themselves in the K-pop culture:\n\n📍 **Cafe de Lulu**\n- **Location:** Gangnam-gu, Seoul\n- **Main Features:** This cafe is decorated with K-pop memorabilia, including posters, albums, and merchandise from various K-pop groups. It has a cozy atmosphere perfect for fans to hang out.\n- **K-Content Connection:** The cafe often plays K-pop music videos and hosts events related to popular idols, making it a great spot for fans to connect with their favorite content.\n- **Recommended For:** K-pop fans looking for a themed dining experience and a place to socialize with fellow enthusiasts.\n- **Nearby Spots:** COEX Mall, a popular shopping and entertainment complex.\n\n📍 **K-Star Road**\n- **Location:** Gangnam-gu, Seoul\n- **Main Features:** This road features a series of K-pop themed cafes and restaurants, each representing different K-pop groups. The area is known 

In [ ]:
### 📍 **진미식당**
- **위치: 서울 마포구 마포대로 186-6, 04207
- **요리:** 간장게장 전문 고급 해산물.
- **설명:** 미슐랭 가이드 서울 2025 등록 레스토랑에서는 서해에서 잡은 게로 만든 신선한 게 요리를 제공합니다. 간장게장은 3일간 발효되어 풍부한 풍미를 보장합니다. 또한 다양한 반찬을 제공하여 현지인과 유명인 모두가 즐겨 찾는 레스토랑입니다.
- **추천 대상:** 해산물 애호가와 독특한 식사 경험을 원하는 분들.

### 📍 **목포항 간장게장 아구침**
- **위치: 서울 서초구 신반포로 47길 06526
- **요리:** 간장게장과 아귀찜을 전문으로 하는 해산물.
- **설명:** 깊고 담백한 간장게장으로 유명한 이 레스토랑은 게장과 아귀찜 등 다양한 게 요리를 제공합니다. 맛이 풍부하고 중독성이 강해 즐거운 식사 경험을 선사합니다.
- **추천 대상:** 게 애호가와 전통 한국 해산물 요리를 맛보고 싶은 분들.

### 📍 **평안 만두집**
- **위치:** 서울 종로구 새문안로3길 03173 30
- **요리:** 전통 한국 만두.
- **설명:** 아늑한 레스토랑에서는 직접 만든 평양식 만두와 보쌈(양파무침 생굴), 진한 녹두전을 제공합니다. 담백한 맛으로 유명한 이 요리는 전통 한국 요리를 즐기는 분들에게 어필할 수 있습니다.
- **추천 대상:** 만두 팬들과 정통 한국식 편안한 음식을 맛보고 싶은 분들.